# Prediction with LSTM model (v4.5.4)
<p style='text-align: right;'>with selectd5.csv</p>

* history
  * 2025/05/23 PM06:02 : 3번째 모델의 3번째 데이터를 사용
  * 2025/05/27
  * 2025/05/30 : v4.5.1 작성 시작
  * 2025/06/04 : v4.5.2 last update
  * 2025/06/04 : v4.5.3 작성
  * 2025/06/05 : v4.5.4 작성

In [8]:
!pip install scikit-learn
!pip install tensorflow
!pip install numpy pandas keras scikit-learn matplotlib scikeras
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
restart kernel... 2025-06-05 13:12:57.373125


In [9]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

restart_kernel()

import os
import sys
from datetime import datetime
cwd = os.getcwd()
sys.path.append(cwd)
print(sys.path)

restart kernel... 2025-06-04 13:35:20.751728
['/home/swhors/jupyter-workspace/LSTM/$PWD', '/home/swhors/jupyter-workspace/finance', '/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-paches', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-packages', '/home/swhors/jupyter-workspace/LSTM']


In [1]:
# load dependacies
import os
import time
import resource
from datetime import datetime
from lib.globalvar import *
from lib.data_loader import DataLoader
from lib.small_predict import predict_small, print_predict_small
from lib.activation import ActivationOutput, RecurrentActivation
from lib.small_predict import build_and_predict, gen_multi_model, save_model, generate_metric
from lib.common_env_sets import lstm_units, dense_units, steps, metrics, dropout
from lib.common_env_sets import learning_rate, last_lstm_return_sequences
from lib.common_env_sets import loss, output_dense_activation, epochs, rand_seed


print(f"imported library. ({datetime.now()})")

imported library. (2025-06-05 10:55:43.949441)


In [2]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./csv/selectd5.csv'
    
mode='predict' # help = "back-test or predict")

mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
training_length=0.95 # default = 0.9)
from_pos = 0
last = [[8, 11, 14, 17, 36, 39],
        [1, 5, 18, 20, 30, 35],
        [7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")


finished to set environemnt. (2025-06-05 10:56:04.107946)


In [3]:
dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0,
                  verbose=False
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300,
                  verbose=False
                  )
    }

print(f'completed to load data. {datetime.now()}')

completed to load data. 2025-06-05 10:56:07.284056


In [4]:
###########
## layers
## LSTM Neural 계층 선언
###########
datasets = [dataset_dicts[1], # 0
            dataset_dicts[2], # 1
           ]

lstm_args = {44: {"model":{"learning_rate": learning_rate[0],
                           "dropout": dropout[0],
                           "steps": steps[0], # last output elements count
                           "metrics": metrics[0],
                           "last_lstm_return_sequences": last_lstm_return_sequences[1],
                           "lstm_units": lstm_units[2],
                           "dense_units": dense_units[2],
                           "output_dense_activation": output_dense_activation[3],
                           "loss": loss[0],
                           "return_state": False,
                           "rand_seed": rand_seed[0] # rand_seed[0]
                           },
                 "train": {"epochs": epochs[3],
                           "is_shuffle": False,
                           "batch": 1,
                           "steps_per_epoch": 0 #100
                           },
                 "dataset": datasets[0]
                 },
             }

print(f'completed to set env for all models. {datetime.now()}')

completed to set env for all models. 2025-06-05 10:56:13.254372


In [5]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
model_version_from = 1
model_version_end = 1
args = {'test_id': 'P4_5_4_1',
        'version': model_version_from,
        'write_to_db': True,
        'last': last,
        'mode2': mode2,
        'gen_num': 5,
        'round_limit': 1000,
        'db_file_name': "./db/metrics.db"
        }
generate_metric(args=args,
                from_version=model_version_from,
                to_version=model_version_end,
                lstm_args=lstm_args)


Test Start: 1~1
CPU 시간: 4.253573 초
메모리 사용량: 611888 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 10:56:17.149979
********************
start to train all models. 2025-06-05 10:56:17.150010
44's training. status=start 2025-06-05 10:56:17.150025
44's training. status=end 2025-06-05 10:57:46.493760
completed to train all models. 2025-06-05 10:57:46.493858
train to create model(1). 2025-06-05 10:57:49.493942
CPU 시간: 247.24722 초
메모리 사용량: 866880 KB


In [6]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
model_version_from = 1
model_version_end = 1
args = {'test_id': 'P4_5_4_2',
        'version': model_version_from,
        'write_to_db': True,
        'last': last,
        'mode2': mode2,
        'gen_num': 1,
        'round_limit': 1000,
        'db_file_name': "./db/metrics.db"
        }
generate_metric(args=args,
                from_version=model_version_from,
                to_version=model_version_end,
                lstm_args=lstm_args)


Test Start: 1~1
CPU 시간: 247.786099 초
메모리 사용량: 866880 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 11:06:32.117262
********************
start to train all models. 2025-06-05 11:06:32.117299
44's training. status=start 2025-06-05 11:06:32.117315
44's training. status=end 2025-06-05 11:08:01.975723
completed to train all models. 2025-06-05 11:08:01.975824
train to create model(1). 2025-06-05 11:08:04.975909
CPU 시간: 491.850567 초
메모리 사용량: 973584 KB


In [7]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
model_version_from = 1
model_version_end = 1
args = {'test_id': 'P4_5_4_3',
        'version': model_version_from,
        'write_to_db': True,
        'last': last,
        'mode2': mode2,
        'gen_num': 1,
        'round_limit': 1000,
        'db_file_name': "./db/metrics.db"
        }
generate_metric(args=args,
                from_version=model_version_from,
                to_version=model_version_end,
                lstm_args=lstm_args)


Test Start: 1~1
CPU 시간: 492.661234 초
메모리 사용량: 973584 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 13:08:23.557416
********************
start to train all models. 2025-06-05 13:08:23.557446
44's training. status=start 2025-06-05 13:08:23.557462
44's training. status=end 2025-06-05 13:09:52.354555
completed to train all models. 2025-06-05 13:09:52.354655
train to create model(1). 2025-06-05 13:09:55.354735
CPU 시간: 737.522399 초
메모리 사용량: 1070352 KB


In [9]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
model_version_from = 1
model_version_end = 1
args = {'test_id': 'P4_5_4_4',
        'version': model_version_from,
        'write_to_db': True,
        'last': last,
        'mode2': mode2,
        'gen_num': 1,
        'round_limit': 1000,
        'db_file_name': "./db/metrics.db"
        }
generate_metric(args=args,
                from_version=model_version_from,
                to_version=model_version_end,
                lstm_args=lstm_args)


Test Start: 1~1
CPU 시간: 738.357368 초
메모리 사용량: 1070352 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 13:24:18.305797
********************
start to train all models. 2025-06-05 13:24:18.305833
44's training. status=start 2025-06-05 13:24:18.305849
44's training. status=end 2025-06-05 13:25:48.471795
completed to train all models. 2025-06-05 13:25:48.471898
train to create model(1). 2025-06-05 13:25:51.471983
CPU 시간: 984.226175 초
메모리 사용량: 1165920 KB


In [10]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
model_version_from = 1
model_version_end = 1
args = {'test_id': 'P4_5_4_5',
        'version': model_version_from,
        'write_to_db': True,
        'last': last,
        'mode2': mode2,
        'gen_num': 1,
        'round_limit': 1000,
        'db_file_name': "./db/metrics.db"
        }
generate_metric(args=args,
                from_version=model_version_from,
                to_version=model_version_end,
                lstm_args=lstm_args)


Test Start: 1~1
CPU 시간: 985.017693 초
메모리 사용량: 1165920 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 13:29:28.732839
********************
start to train all models. 2025-06-05 13:29:28.732874
44's training. status=start 2025-06-05 13:29:28.732891
44's training. status=end 2025-06-05 13:30:58.224392
completed to train all models. 2025-06-05 13:30:58.224494
train to create model(1). 2025-06-05 13:31:01.224577
CPU 시간: 1231.022737 초
메모리 사용량: 1256544 KB


In [11]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
model_version_from = 1
model_version_end = 1
args = {'test_id': 'P4_5_4_6',
        'version': model_version_from,
        'write_to_db': True,
        'last': last,
        'mode2': mode2,
        'gen_num': 1,
        'round_limit': 1000,
        'db_file_name': "./db/metrics.db"
        }
generate_metric(args=args,
                from_version=model_version_from,
                to_version=model_version_end,
                lstm_args=lstm_args)


Test Start: 1~1
CPU 시간: 1231.857141 초
메모리 사용량: 1256544 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 13:33:56.449075
********************
start to train all models. 2025-06-05 13:33:56.449098
44's training. status=start 2025-06-05 13:33:56.449108
44's training. status=end 2025-06-05 13:35:27.307357
completed to train all models. 2025-06-05 13:35:27.307456
train to create model(1). 2025-06-05 13:35:30.307540
CPU 시간: 1477.436818 초
메모리 사용량: 1358384 KB


In [12]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
model_version_from = 1
model_version_end = 1
args = {'test_id': 'P4_5_4_7',
        'version': model_version_from,
        'write_to_db': True,
        'last': last,
        'mode2': mode2,
        'gen_num': 1,
        'round_limit': 1000,
        'db_file_name': "./db/metrics.db"
        }
generate_metric(args=args,
                from_version=model_version_from,
                to_version=model_version_end,
                lstm_args=lstm_args)


Test Start: 1~1
CPU 시간: 1478.442048 초
메모리 사용량: 1358384 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 13:51:51.384109
********************
start to train all models. 2025-06-05 13:51:51.384142
44's training. status=start 2025-06-05 13:51:51.384158
44's training. status=end 2025-06-05 13:53:24.419159
completed to train all models. 2025-06-05 13:53:24.419260
train to create model(1). 2025-06-05 13:53:27.419344
CPU 시간: 1724.437278 초
메모리 사용량: 1462752 KB


In [13]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#
model_version_from = 1
model_version_end = 1
args = {'test_id': 'P4_5_4_8',
        'version': model_version_from,
        'write_to_db': True,
        'last': last,
        'mode2': mode2,
        'gen_num': 1,
        'round_limit': 1000,
        'db_file_name': "./db/metrics.db"
        }
generate_metric(args=args,
                from_version=model_version_from,
                to_version=model_version_end,
                lstm_args=lstm_args)


Test Start: 1~1
CPU 시간: 1725.531982 초
메모리 사용량: 1462752 KB
generate_metric.02 args["version"]=1
********************
working model_version = 1  2025-06-05 14:11:06.995248
********************
start to train all models. 2025-06-05 14:11:06.995283
44's training. status=start 2025-06-05 14:11:06.995298
44's training. status=end 2025-06-05 14:12:40.633690
completed to train all models. 2025-06-05 14:12:40.633791
train to create model(1). 2025-06-05 14:12:43.633875
CPU 시간: 1970.768192 초
메모리 사용량: 1560064 KB


In [ ]:
## end